# Fake News Detection

## Import Libraries

In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import re
import string

## Import Dataset

In [3]:
df_fake = pd.read_csv("D:/Fake News Detection/Fake.csv")
df_true = pd.read_csv("D:/Fake News Detection/True.csv")
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


## Incoding the Binary Targets

In [5]:
df_fake["class"] = 0
df_true["class"] = 1

In [6]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

## Merge Dataframes

In [7]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


## Remove Unwanted Cols

In [8]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [9]:
df = df_merge.drop(["title", "subject","date"], axis = 1)
df.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


## Random Shuffling

In [14]:
df = df.sample(frac = 1)
df.head()

,text,class
6232,NEW YORK (Reuters) - Puerto Rico’s federal ove...,1
16088,Legal Expert Jonathan Turley listened as Joe S...,0
10533,(Reuters) - Kentucky’s Republican Governor Mat...,1
12696,The Ecuadorian government admitted on Tuesday ...,0
19059,Van Jones praised President Trump for honoring...,0


In [15]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)
df.head()

,text,class
0,NEW YORK (Reuters) - Puerto Rico’s federal ove...,1
1,Legal Expert Jonathan Turley listened as Joe S...,0
2,(Reuters) - Kentucky’s Republican Governor Mat...,1
3,The Ecuadorian government admitted on Tuesday ...,0
4,Van Jones praised President Trump for honoring...,0


## Check for Null

In [16]:
df.isnull().sum()

text     0
class    0
dtype: int64

## Cleaning and tokenization Function

In [17]:
def preprocess(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [18]:
df["text"] = df["text"].apply(preprocess)
df.head()

,text,class
0,new york reuters puerto rico s federal ove...,1
1,legal expert jonathan turley listened as joe s...,0
2,reuters kentucky s republican governor mat...,1
3,the ecuadorian government admitted on tuesday ...,0
4,van jones praised president trump for honoring...,0


## Define x for features and y for target

In [19]:
x = df["text"]
y = df["class"]

## Train Test Split

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

## Features Extraction Using BOW-TFIDF

In [23]:
tfidf = TfidfVectorizer()
x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test) 

## Logistic Regression

In [25]:
LR = LogisticRegression()
LR.fit(x_train_tfidf,y_train)

LogisticRegression()

**Logistic Regression Evaluation**

In [26]:
pred_lr=LR.predict(x_test_tfidf)
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5887
           1       0.98      0.99      0.99      5338

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



## Decision Tree

In [28]:
DT = DecisionTreeClassifier()
DT.fit(x_train_tfidf, y_train)

DecisionTreeClassifier()

**Decision Tree Evaluation**

In [29]:
pred_dt = DT.predict(x_test_tfidf)
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5887
           1       1.00      0.99      1.00      5338

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



## Ensemble Methods

### 1- Random Forest

In [31]:
RF = RandomForestClassifier(random_state=0)
RF.fit(x_train_tfidf, y_train)

RandomForestClassifier(random_state=0)

**Evaluaiton**

In [32]:
pred_rf = RF.predict(x_test_tfidf)
print(classification_report(y_test, pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5887
           1       0.99      0.99      0.99      5338

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



### 2- Gradient Boosting

In [34]:
GB = GradientBoostingClassifier(random_state=0)
GB.fit(x_train_tfidf, y_train)

GradientBoostingClassifier(random_state=0)

**Evaluation**

In [35]:
pred_gb = GB.predict(x_test_tfidf)
print(classification_report(y_test, pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5887
           1       0.99      1.00      1.00      5338

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



# The End!
## Best Regard,
### Ibrahim M. Nasser